In [1]:
import ipywidgets as widgets
from traitlets import Unicode, List, Int

In [2]:
widgets.__version__

'7.0.0'

# Example from https://bl.ocks.org/mbostock/3885304


In [3]:
%%javascript

var s = document.createElement("style");
s.innerHTML = `
.bar {
  fill: steelblue;
}

.bar:hover {
  fill: brown;
}

.axis--x path {
  display: none;
}
`;
document.getElementsByTagName("head")[0].appendChild(s);

<IPython.core.display.Javascript object>

In [4]:
class MyD3(widgets.DOMWidget):
    _view_name = Unicode('D3View').tag(sync=True)
    _view_module = Unicode('myd3').tag(sync=True)
    width = Int().tag(sync=True)
    height = Int().tag(sync=True)
    vertices = List().tag(sync=True)

In [4]:
from io import StringIO
import pandas as pd


In [17]:
text = StringIO("""
letter	frequency
A	.08167
B	.01492
C	.02782
D	.04253
E	.12702
F	.02288
G	.02015
H	.06094
I	.06966
J	.00153
K	.00772
L	.04025
M	.02406
N	.06749
O	.07507
P	.01929
Q	.00095
R	.05987
S	.06327
T	.09056
U	.02758
V	.00978
W	.02360
X	.00150
Y	.01974
Z	.00074
""")

In [18]:
df = pd.read_table(text)

In [19]:
df.head()

,letter,frequency
0,A,0.08167
1,B,0.01492
2,C,0.02782
3,D,0.04253
4,E,0.12702


In [8]:
%%javascript
require.undef('myd3');

define('myd3', ["@jupyter-widgets/base", //revised after upgrade to ipywidget 7.0s
                "https://d3js.org/d3.v4.min.js"], function(widgets, d3) {

    var D3View = widgets.DOMWidgetView.extend({

        render: function() {

            var that = this;

            var width = this.model.get('width'),
                height = this.model.get('height');

            that.sites = this.model.get('vertices');    
            that.svg = d3.select(this.el).append("svg")
                                         .attr("width", width)
                                         .attr("height", height)
            
            var margin = {top: 20, right: 20, bottom: 30, left: 40};
            width = width - margin.left - margin.right;
            height = height - margin.top - margin.bottom;
            
            var x = d3.scaleBand().rangeRound([0, width]).padding(0.1),
                y = d3.scaleLinear().rangeRound([height, 0]);

            var g = svg.append("g")
                        .attr("transform", "translate(" + margin.left + "," + margin.top + ")");
            
            d3.tsv("data.tsv", function(d) {
                  d.frequency = +d.frequency;
                  return d;
                }, function(error, data) {
                  if (error) throw error;

            that.voronoi = d3.voronoi().extent([[-1, -1], [width + 1, height + 1]]);
            that.polygon = that.svg.append("g")
                                    .attr("class", "polygons")
                                  .selectAll("path")
                                  .data(that.voronoi.polygons(that.sites))
                                  .enter().append("path")
                                    .call(that.redrawPolygon);

            that.link = that.svg.append("g")
                                .attr("class", "links")
                              .selectAll("line")
                              .data(that.voronoi.links(that.sites))
                              .enter().append("line")
                                .call(that.redrawLink);

            that.site = that.svg.append("g")
                                .attr("class", "sites")
                              .selectAll("circle")
                              .data(that.sites)
                              .enter().append("circle")
                                .attr("r", 2.5)
                                .call(that.redrawSite);            
            that.model.on('change:vertices', that.redraw, this);
            that.redraw();
        },

        redraw: function() {
          this.sites = this.model.get('vertices');
          var diagram = this.voronoi(this.sites);
          this.polygon = this.polygon.data(diagram.polygons()).call(this.redrawPolygon);
          this.link = this.link.data(diagram.links());
          this.link.exit().remove();
          this.link = this.link.enter().append("line").merge(this.link).call(this.redrawLink);
          this.site = this.site.data(this.sites).call(this.redrawSite);
        },

        redrawPolygon: function(polygon) {
          polygon
              .attr("d", function(d) { return d ? "M" + d.join("L") + "Z" : null; });
        },

        redrawLink: function(link) {
          link
              .attr("x1", function(d) { return d.source[0]; })
              .attr("y1", function(d) { return d.source[1]; })
              .attr("x2", function(d) { return d.target[0]; })
              .attr("y2", function(d) { return d.target[1]; });
        },
        redrawSite: function(site) {
          site
              .attr("cx", function(d) { return d[0]; })
              .attr("cy", function(d) { return d[1]; });
        }

    });

    return {
        D3View : D3View
    };
});

<IPython.core.display.Javascript object>

In [9]:
import numpy as np
sample_size = 100
width = 750
height = 300

m = MyD3(vertices=(np.random.rand(sample_size, 2) * np.array([width, height])).tolist(),
     height=height, width=width)
m

A Jupyter Widget

In [12]:
m.vertices = (np.random.rand(sample_size, 2) * np.array([width, height])).tolist()